# Example api

In [1]:
from adelecv.api.logs import enable_logs
from adelecv.api.data.segmentations import SegmentationDataset
from logging import StreamHandler

To enable logs, there is a function
 ```python
enable logs
 ```
You can pass any necessary logging hundler

In [2]:
enable_logs(StreamHandler())

In [3]:
import torch
torch.cuda.is_available()

True

In [4]:
from adelecv.api.config import Settings
from pathlib import Path

In [5]:
# you can change the path to save logs and weights
# Settings.update_tmp_path(Path('your_path'))

In [6]:
from adelecv.api.config import Settings

In [7]:
Settings.TMP_PATH

WindowsPath('tmp/9fa1cc64111e44bb9dfe9650797dcf25')

1. You need to create a dataset and pass the necessary parameters to it

In [8]:
from adelecv.api.data.segmentations.types import ImageMask

In [9]:
segm_dataset = SegmentationDataset(
    r'F:\dataset\ph2',  # path to dataset
    ImageMask,          # dataset type
    (256, 256),         # img size HxW
    (0.7, 0.2, 0.1),    # train val test split
    16,                 # batch size
)

 100% |█████████████████| 200/200 [1.3s elapsed, 0s remaining, 152.9 samples/s]         


INFO - Split dataset train size: 140, valid size: 40, test size: 20
INFO - Creating a dataset
DEBUG - Dataset created with params, dataset dir: F:\dataset\ph2, classes: {0: 'background', 1: 'target'}, batch size: 16


To get possible values for hyperparameters, there are functions presented below

In [10]:
from adelecv.api.models.segmentations import  get_encoders, get_pretrained_weights, get_models, get_torch_optimizers, get_losses, get_optimize_scores
from adelecv.api.optimize.segmentations import get_hp_optimizers
# создать отдельный модуль для получения возможных параметров

In [11]:
print('models: ', get_models())
print('encoders: ', get_encoders())
print('pretrained_weights', get_pretrained_weights())
print('optimizers', get_torch_optimizers())
print('losses', get_losses())
print('hp optimizers', get_hp_optimizers())
print('optimize scores', get_optimize_scores())

models:  ['Unet', 'UnetPlusPlus', 'MAnet', 'Linknet', 'FPN', 'PSPNet', 'DeepLabV3', 'DeepLabV3Plus', 'PAN']
encoders:  ['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'resnext50_32x4d', 'resnext101_32x4d', 'resnext101_32x8d', 'resnext101_32x16d', 'resnext101_32x32d', 'resnext101_32x48d', 'dpn68', 'dpn68b', 'dpn92', 'dpn98', 'dpn107', 'dpn131', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn', 'vgg19', 'vgg19_bn', 'senet154', 'se_resnet50', 'se_resnet101', 'se_resnet152', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'inceptionresnetv2', 'inceptionv4', 'efficientnet-b0', 'efficientnet-b1', 'efficientnet-b2', 'efficientnet-b3', 'efficientnet-b4', 'efficientnet-b5', 'efficientnet-b6', 'efficientnet-b7', 'mobilenet_v2', 'xception', 'timm-efficientnet-b0', 'timm-efficientnet-b1', 'timm-efficientnet-b2', 'timm-efficientnet-b3', 'timm-efficientnet-b4', 'timm-efficientnet-b5', 'timm-efficientnet-b6', 'timm-e

2. It is necessary to create a set of hyperparameters, according to which the optimal ones will be searched.

In [12]:
from adelecv.api.optimize.segmentations import HyperParamsSegmentation
from adelecv.api.optimize.segmentations import HPOptimizer

In [13]:
params = HyperParamsSegmentation(
    strategy='TPESampler',
    architectures=['UnetPlusPlus'],
    encoders=['mobilenet_v2'],
    pretrained_weights=['imagenet'],
    loss_fns=['JaccardLoss'],
    optimizers=['AdamW', 'RMSprop'],
    epoch_range=(5, 5),
    lr_range=(0.001, 0.003),
    optimize_score='loss',
)

3. You need to create an optimizer object with the transfer of the dataset and hyperparameters to it. You also need to choose an optimization algorithm and set additional training parameters

In [15]:
hp_optimizer = HPOptimizer(
    hyper_params=params,
    num_trials=3,
    device='GPU',
    dataset=segm_dataset
)

DEBUG - Create hp optimizer with params: strategy: TPESampler num_trials: 1 num_classes: 2 device: cuda hps: {'strategy': 'TPESampler', 'architectures': ['UnetPlusPlus'], 'encoders': ['mobilenet_v2'], 'pretrained_weights': ['imagenet'], 'loss_fns': ['JaccardLoss'], 'optimizers': ['AdamW', 'RMSprop'], 'epoch_range': (5, 5), 'lr_range': (0.001, 0.003), 'optimize_score': 'loss'}
DEBUG - Create hp optimizer with params: strategy: TPESampler num_trials: 3 num_classes: 2 device: cuda hps: {'strategy': 'TPESampler', 'architectures': ['UnetPlusPlus'], 'encoders': ['mobilenet_v2'], 'pretrained_weights': ['imagenet'], 'loss_fns': ['JaccardLoss'], 'optimizers': ['AdamW', 'RMSprop'], 'epoch_range': (5, 5), 'lr_range': (0.001, 0.003), 'optimize_score': 'loss'}


4. Tensorboard is used to visualize learning

In [16]:
from adelecv.api.config import Settings

In [17]:
tensorboard_logs_path = Settings.TENSORBOARD_LOGS_PATH.as_posix()

In [17]:
tensorboard_logs_path

'tmp/cb0cb0afd3f84fb6b0d8299532ad8759/tensorboard'

In [18]:
%reload_ext tensorboard

In [19]:
tensorboard_logs_path

'tmp/cb0cb0afd3f84fb6b0d8299532ad8759/tensorboard'

In [22]:
tensorboard --logdir $tensorboard_logs_path

^C


Launching TensorBoard...

5. To start the training, call the appropriate method.

In [17]:
hp_optimizer.optimize()

INFO - Train models started
[I 2023-05-01 17:46:09,162] A new study created in memory with name: no-name-f6be76f6-855f-4d0f-960b-047ecab722b2
DEBUG - Create tensorboard logger, path: tmp/9fa1cc64111e44bb9dfe9650797dcf25/tensorboard/499ba36
DEBUG - Dataset updated
INFO - Model 499ba36_UnetPlusPlus_mobilenet_v2_imagenet_AdamW_JaccardLoss_lr=0,0022375857163229526 trained with test loss 0.14925605058670044
DEBUG - Add predictions for model: 499ba36_UnetPlusPlus_mobilenet_v2_imagenet_AdamW_JaccardLoss_lr=0,0022375857163229526


 200 [9.4s elapsed, 20.8 samples/s]   


DEBUG - Save weights model: 499ba36_UnetPlusPlus_mobilenet_v2_imagenet_AdamW_JaccardLoss_lr=0,0022375857163229526, path: tmp/9fa1cc64111e44bb9dfe9650797dcf25/weights/499ba36.pt
[I 2023-05-01 17:47:01,928] Trial 0 finished with value: 0.11668000618616739 and parameters: {'optimizer': 'AdamW', 'architecture': 'UnetPlusPlus', 'encoders': 'mobilenet_v2', 'pretrained_weight': 'imagenet', 'lr': 0.0022375857163229526, 'loss': 'JaccardLoss', 'num_epoch': 5}. Best is trial 0 with value: 0.11668000618616739.
DEBUG - Create tensorboard logger, path: tmp/9fa1cc64111e44bb9dfe9650797dcf25/tensorboard/d81e347
DEBUG - Dataset updated
INFO - Model d81e347_UnetPlusPlus_mobilenet_v2_imagenet_AdamW_JaccardLoss_lr=0,0020570884365662253 trained with test loss 0.13842931389808655
DEBUG - Add predictions for model: d81e347_UnetPlusPlus_mobilenet_v2_imagenet_AdamW_JaccardLoss_lr=0,0020570884365662253


 200 [12.1s elapsed, 15.4 samples/s]  


DEBUG - Save weights model: d81e347_UnetPlusPlus_mobilenet_v2_imagenet_AdamW_JaccardLoss_lr=0,0020570884365662253, path: tmp/9fa1cc64111e44bb9dfe9650797dcf25/weights/d81e347.pt
[I 2023-05-01 17:47:54,924] Trial 1 finished with value: 0.10880672931671143 and parameters: {'optimizer': 'AdamW', 'architecture': 'UnetPlusPlus', 'encoders': 'mobilenet_v2', 'pretrained_weight': 'imagenet', 'lr': 0.0020570884365662253, 'loss': 'JaccardLoss', 'num_epoch': 5}. Best is trial 1 with value: 0.10880672931671143.
DEBUG - Create tensorboard logger, path: tmp/9fa1cc64111e44bb9dfe9650797dcf25/tensorboard/304731e
DEBUG - Dataset updated
INFO - Model 304731e_UnetPlusPlus_mobilenet_v2_imagenet_AdamW_JaccardLoss_lr=0,0025402917390305577 trained with test loss 0.09902772307395935
DEBUG - Add predictions for model: 304731e_UnetPlusPlus_mobilenet_v2_imagenet_AdamW_JaccardLoss_lr=0,0025402917390305577


 200 [15.2s elapsed, 11.6 samples/s]  


DEBUG - Save weights model: 304731e_UnetPlusPlus_mobilenet_v2_imagenet_AdamW_JaccardLoss_lr=0,0025402917390305577, path: tmp/9fa1cc64111e44bb9dfe9650797dcf25/weights/304731e.pt
[I 2023-05-01 17:48:53,994] Trial 2 finished with value: 0.08640851577123006 and parameters: {'optimizer': 'AdamW', 'architecture': 'UnetPlusPlus', 'encoders': 'mobilenet_v2', 'pretrained_weight': 'imagenet', 'lr': 0.0025402917390305577, 'loss': 'JaccardLoss', 'num_epoch': 5}. Best is trial 2 with value: 0.08640851577123006.
INFO - Study statistics:
INFO - Number of finished trials: 3
INFO - Number of pruned trials: 0
INFO - Number of complete trials: 3
INFO - Best trial:
INFO - Value: 0.08640851577123006
INFO - Params: 
INFO - optimizer: AdamW
INFO - architecture: UnetPlusPlus
INFO - encoders: mobilenet_v2
INFO - pretrained_weight: imagenet
INFO - lr: 0.0025402917390305577
INFO - loss: JaccardLoss
INFO - num_epoch: 5
INFO - Train models is over


In [19]:
hp_optimizer.stats_models

,_id,name,architecture,encoder,pretrained_weight,lr,optimizer,loss_fn,num_epoch,loss,fbeta_score,f1_score,iou_score,accuracy,positive_predictive_value,sensitivity
0,499ba36,499ba36_UnetPlusPlus_mobilenet_v2_imagenet_Ada...,UnetPlusPlus,mobilenet_v2,imagenet,0.002238,AdamW,JaccardLoss,5,0.149,0.868,0.868,0.810,0.931,0.900,0.901
1,d81e347,d81e347_UnetPlusPlus_mobilenet_v2_imagenet_Ada...,UnetPlusPlus,mobilenet_v2,imagenet,0.002057,AdamW,JaccardLoss,5,0.138,0.876,0.876,0.823,0.934,0.890,0.929
2,304731e,304731e_UnetPlusPlus_mobilenet_v2_imagenet_Ada...,UnetPlusPlus,mobilenet_v2,imagenet,0.002540,AdamW,JaccardLoss,5,0.099,0.928,0.928,0.884,0.952,0.935,0.944


6 (Optional). A special class is provided for exporting weights (creating a zip archive). Mainly used for the web version of the library

In [25]:
from adelecv.api.modification_models import ExportWeights

In [ ]:
# you can save the selected weights in a zip file (feature for ui)
# ExportWeights().create_zip()

7. You can visualize the datasets and prediction of each model using the fiftyone.

In [26]:
import fiftyone as fo

In [32]:
fo.launch_app(segm_dataset.fo_dataset)

Dataset:          ImageMask
Media type:       image
Num samples:      200
Selected samples: 0
Selected labels:  0
Session URL:      http://localhost:5151/

8. To convert weights, you can use the appropriate classes. Below is an example for the onnx format

In [22]:
from adelecv.api.modification_models import ConvertWeights

In [23]:
cw = ConvertWeights(input_shape=(1, 3, 256, 256))

In [24]:
cw.run(id_selected=['304731e', '499ba36'], new_format='onnx')

INFO - Сonvert weights model: 304731e to onnx format
D:\anaconda\envs\adelecv-check\lib\site-packages\segmentation_models_pytorch\base\model.py:16: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if h % output_stride != 0 or w % output_stride != 0:
D:\anaconda\envs\adelecv-check\lib\site-packages\torch\onnx\symbolic_helper.py:829: UserWarning: You are trying to export the model with onnx:Resize for ONNX opset version 10. This operator might cause results to not match the expected results by PyTorch.
ONNX's Upsample/Resize operator did not match Pytorch's Interpolation until opset 11. Attributes to determine how to transform the input were added in onnx:Resize in opset 11 to support Pytorch's behavior (like coordinate_transformation_mode and nearest_mode).
We recommend us

============= Diagnostic Run torch.onnx.export version 2.0.0+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



INFO - Create zip with converted weights, path: tmp/9fa1cc64111e44bb9dfe9650797dcf25/converted_bdb48d4c20604da9912eb70d68588744.zip
INFO - Сonvert weights model: 499ba36 to onnx format


============= Diagnostic Run torch.onnx.export version 2.0.0+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



INFO - Create zip with converted weights, path: tmp/9fa1cc64111e44bb9dfe9650797dcf25/converted_bdb48d4c20604da9912eb70d68588744.zip


WindowsPath('tmp/9fa1cc64111e44bb9dfe9650797dcf25/converted_bdb48d4c20604da9912eb70d68588744.zip')